# TDSE with MPI

Here you will learn how to speed up you `TDSE` code using MPI. First a new-format Richmol file, containing energies of field-free states and matrix elements of field tensor operators for water, is downloaded from `zenodo`:

In [1]:
import urllib.request
import os
import h5py

# GET RICHMOL FILE
richmol_file = "h2o_p48_j40_emax4000_rovib.h5"
if not os.path.exists(richmol_file):
    url = "https://zenodo.org/record/4986069/files/h2o_p48_j40_emax4000_rovib.h5"
    print(f"download richmol file from {url}")
    urllib.request.urlretrieve(url, "h2o_p48_j40_emax4000_rovib.h5")
    print("download complete")

This new-format Richmol file contains following datasets:

In [2]:
with h5py.File(richmol_file, "r") as fl:
    print("Available datasets")
    for key, val in fl.items():
        print(f"'{key}'")                   # dataset name
        print("\t", val.attrs["__doc__"])   # dataset description

Available datasets
'dipole'
	 Cartesian tensor operator, store date: 2021-06-18 17:57:50, comment: electric dipole moment in Debye, dipole moment surface: [DOI:10.1063/1.5043545]
'h0'
	 Cartesian tensor operator, store date: 2021-06-18 17:56:49, comment: Authors: Andrey Yachmenev (CFEL/DESY) and Sergei Yurchenko (UCL/London). TROVE variationally calculated of ro-vibrational energies and transitions for water H_2^16O. The data is truncated at energies below 4000 cm^-1. Basis: P48, J: 0..40, PES: [DOI:10.1098/rsta.2017.0149]. Calculated ro-vibrational energies substituted by empirical values from [DOI:10.1016/j.jqsrt.2012.10.002] where available. This data was produced in the following works [DOI:10.1103/PhysRevResearch.2.023091], [DOI:10.1039/D0CP01667E]. Notes on state assignment: the k-group rovibrational assignment is given by a string containing rovibrational quanta in the following order 'k tau v1 v2 v3 rot_sym vib_sym repl', where k and tau are rotational quantum numbers, v1, v2, 

Consider the interaction between electric dipole moment and external dc field:

In [24]:
from richmol.field import CarTens
from richmol.convert_units import Debye_x_Vm_to_invcm

# INITIALIZE CARTESIAN TENSOR OPERATORS
def filt(**kwargs):
    pass_J, pass_enr = True, True
    if 'J' in kwargs:
        pass_J = kwargs['J'] <= 5
    if 'enr' in kwargs:
        pass_enr = kwargs['enr'] <= 1e3
    return pass_J and pass_enr
h0 = CarTens(richmol_file, name='h0', bra=filt, ket=filt)
mu = CarTens(richmol_file, name='dipole', bra=filt, ket=filt) \
    * (-0.5 * Debye_x_Vm_to_invcm())

# INITIALIZE EXTERNAL DC FIELD
field = lambda t : [0, 0, t * 1e5 ] # (V/m)

## Time-propagation without MPI

A `TDSE` solver and initial state vectors, spanning the space of field-free states, are initialized:

In [36]:
from richmol.tdse import TDSE
import time

# INITIALIZE TDSE AND STATES
tdse = TDSE(t_start=0, t_end=1, dt=0.1, t_units="ps", enr_units="invcm")
vecs = tdse.init_state(h0, temp=None)
print('number of states to propagate : ', vecs.shape[0])

number of states to propagate :  286


The initial states are propagated in time:

In [37]:
time_evolution = [vecs]
time_0 = time.time()
for i, t in enumerate(tdse.time_grid()):
    vecs_, _ = tdse.update(
        mu*field(t), H0=h0, vecs=time_evolution[i], matvec_lib='scipy'
    )
    time_evolution.append(vecs_)
runtime = time.time() - time_0
print('runtime without MPI : {} sec'.format(round(runtime, 3)))

runtime without MPI : 24.2 sec


## Time-propagation with MPI

A `TDSE` solver is initialized (the previously computed initial state vectors are used):

In [40]:
# INITIALIZE TDSE
tdse = TDSE(t_start=0, t_end=1, dt=0.1, t_units="ps", enr_units="invcm")

The intial states are propagated in time:

In [46]:
%%px

from mpi4py import MPI

# INITIALIZE MPI VARIABLES
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
world_size = comm.Get_size()
print('world size : ', world_size)

UsageError: Cell magic `%%px` not found.
